In [34]:
import datetime
import json
import hashlib
from flask import Flask, request, jsonify
from uuid import uuid4
from urllib.parse import urlparse
import requests

In [35]:
class BhattiChain:
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.nodes = set()
        self.addBlock(proof = 1, prevHash= "0")
    
    def addBlock(self, proof, prevHash):
        block = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash,
            "transactions": self.transactions 
        }
        self.transactions = []
        self.chain.append(block)
        return block
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encodedBlock).hexdigest()
    
    def proofOfWork(self, prevProof):
        new_proof = 1
        solved = False
        
        while solved is False:
            hashProblem = hashlib.sha256(str(new_proof **2 - prevProof ** 2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    
    def isChainValid(self, chain):
        previousBlock = chain[0]
        currentIndex = 1
        
        while currentIndex < len(chain):
            currentBlock = chain[currentIndex]
            currentPrevHash = currentBlock["prevHash"]
            if currentPrevHash != self.hashBlock(previousBlock):
                return False
            currentProof = currentBlock["proof"]
            previousProof = previousBlock["proof"]
            hashProblem = hashlib.sha256(str(currentProof **2 - previousProof ** 2).encode()).hexdigest()
            if hashProblem[:4] != "0000":
                return False
            previousBlock = currentBlock
            currentIndex += 1
        return True
    
    def addTransaction(self, sender, receiver, amount):
        self.transactions.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        latestBlock = self.getLastBlock()
        return latestBlock["index"] + 1
    
    def addNode(self, ipAddress):
        nodeIp = urlparse(ipAddress)
        self.nodes.add(nodeIp.netloc)
    
    def shouldReplaceChain(self):
        network = self.nodes
        largestChain = None
        maxChainLength = len(self.chain)
        
        for node in network:
            response = requests.get(f"http://{node}/chain")
            if response.status_code == 200:
                length = response.json()["length"]
                chain = response.json()["chain"]
                if length > maxChainLength and self.isChainValid(chain):
                    largestChain = chain
                    maxChainLength = length
            if largestChain:
                self.chain =largestChain
                return True
        return False    

In [36]:
bhatti  = BhattiChain()

In [37]:
nodeAddress = str(uuid4()).replace("-", "")

In [38]:
app = Flask(__name__)

In [39]:
@app.route("/transaction", methods = ["POST"])
def transaction():
    transJson = request.get_json()
    transKeys = ["sender", "receiver", "amount"]
    if not all (key in transJson for key in transKeys):
        return "There are some keys missing"
    index = bhatti.addTransaction(transJson["sender"], transJson["receiver"], transJson["amount"])
    response = {
        "success": f"The transaciton will be added to the next block at index {index}"
    }
    return jsonify(response), 201

In [40]:
@app.route("/connect", methods = ["POST"])
def connect():
    nodeJson = request.get_json()
    nodes = nodeJson.get("nodes")
    if nodes is None:
        return "Nodes are empty"
    for node in nodes:
        bhatti.addNode(node)
    response = {
        "success": "All nodes have been added",
        "length": list(bhatti.nodes)
    }
    return jsonify(response), 201

In [41]:
@app.route("/replacechain", methods = ["GET"])
def replacechain():
    replace = bhatti.shouldReplaceChain()
    if replace:
        response = {
            "message":"Chain will be replaced since we found a larger",
            "chain": bhatti.chain,
            "chainLength": len(bhatti.chain)
        }
    else:
         response = {
            "message":"Chain was fine tere werent any changes ",
            "chain": bhatti.chain,
            "chainLength": len(bhatti.chain)
        }
    return jsonify(response), 200
        

In [42]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": bhatti.chain,
        "length": len(bhatti.chain)
    }
    return jsonify(response), 200

In [43]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousBlock = bhatti.getLastBlock()
    previousProof = previousBlock["proof"]
    proof = bhatti.proofOfWork(previousProof)
    previousHash = bhatti.hashBlock(previousBlock)
    bhatti.addTransaction(sender=nodeAddress, receiver ="Berkat", amount = 10)
    block = bhatti.addBlock(proof, previousHash)
    response = {
        "success": "you have successfully mined a block!!!!!",
        "index": block["index"],
        "proof": block["proof"],
        "datetime": block["timestamp"]
    }
    return jsonify(response), 200

In [ ]:
app.run(host = "0.0.0.0", port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2021 00:47:56] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:00:24] "POST /connect HTTP/1.1" 201 -
127.0.0.1 - - [30/Dec/2021 01:00:58] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:01:00] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:01:01] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:01:04] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:01:29] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:03:02] "POST /transaction HTTP/1.1" 201 -
127.0.0.1 - - [30/Dec/2021 01:03:19] "POST /transaction HTTP/1.1" 201 -
127.0.0.1 - - [30/Dec/2021 01:03:43] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:03:50] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2021 01:04:19] "GET /chain HTTP/1.1" 200 -
